In [1]:
import sys
sys.path.append("../script/")

import Functions
from Engine import Engine
from GTGP import GTGP


from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import importlib
from time import time
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [2]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """
    
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    print(roc_auc_score(true_y,prob[:,1]))

# data

In [3]:
df = pd.read_csv("../data/kddcup.tsv",delimiter='\t')

X = df.iloc[:,:-1].to_numpy().astype("float")
y = df.iloc[:,-1].to_numpy().astype("int")

seeds = [10086, 200,500,30506,30405,30420,10056,7059,40965,5398,869543,83491,823190,
         48392,2810,48392,3498210,483902,859032,12890,538920,86954,54309,6504,9840,
         219805,548,2981,432890,5438908,219094,5843902,60854,979,12890,2108,4093]
# seeds = seeds[::-1]
train_size = 0.7
dataset = 'kdd99'

In [4]:
def to_one_hot(y,yt):
    le = LabelEncoder()
    ohe = OneHotEncoder()
    y_train = le.fit_transform(y)
    y_one_hot = ohe.fit_transform(y_train.reshape(-1,1))
    
    y_test = le.transform(yt)
    yt_one_hot = ohe.transform(y_test.reshape(-1,1))
    
    return y_train,y_test,y_one_hot,yt_one_hot

# model

In [7]:
def fit_trees():
    learning_rate=0.1
    max_depth=3
    bins=8
    lam=1

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size=10
    elite_size = 3
    epoch= 1
    gp_epoch=1000
    verbose = 1
    tolerance=0.01

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    return gtgp

def retrain(gtgp):
    retrain_epoch= 10
    alpha=1
    beta=1
    gammer=1

    verbose=1
    gtgp.lam = 10
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

for i in range(30,13,-1):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    gtgp = fit_trees()
    retrain(gtgp)

    print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
    print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))
    print(gtgp.test_sse)

    num_trees,depth,num_nodes = gtgp.print_model()

    train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
    test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
    train_f1 = f1_score(y_train,np.argmax(gtgp.train_p,axis=1),average='macro')
    test_f1 = f1_score(y_test,np.argmax(gtgp.test_p,axis=1),average='macro')

    break
    # with open('./benchmark/'+dataset+'.csv','a') as f:
    #     s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
    #     f.writelines(s)

2250 769876 1
0.26997947692871094
0.3067162036895752
0.31784510612487793
0.27877235412597656
0.26410961151123047
0.2353222370147705
0.25757551193237305
0.2793118953704834
0.2601163387298584
0.25176048278808594
0.2599802017211914
0.26425695419311523
0.24515795707702637
0.2643871307373047
0.29140472412109375
0.2968730926513672
0.273484468460083
0.2679150104522705
0.25411033630371094
0.2852213382720947
0.2845289707183838
0.24621915817260742
0.26792311668395996
0.28154635429382324
0.26499295234680176
0.27072930335998535
0.299774169921875
0.2609257698059082
0.24541306495666504
0.24225449562072754
0.2596771717071533
0.2481069564819336
0.23688340187072754
0.22873377799987793
0.2274184226989746
0.23608183860778809
0.2548387050628662
0.230621337890625
0.2288036346435547
0.25391578674316406
0.23801469802856445
0.23327302932739258
0.2597064971923828
0.2333519458770752
0.23716974258422852
0.2585330009460449
0.24432849884033203
0.2414696216583252
0.23484516143798828
0.2464916706085205
0.23499178886

KeyboardInterrupt: 

In [ ]:
gtgp.test_sse

[0.7332723013620768,
 0.7803155234203581,
 0.804044530937759,
 0.8167639566541077,
 0.8168353923943468,
 0.8646614793508683,
 0.8646600589964869,
 0.8646600589964869,
 0.8646600589964869,
 0.8647068105862672]

In [ ]:
retrain_epoch=10
alpha=1
beta=1
gammer=0

verbose=1
gtgp.lam = 10
train_acc,test_acc,train_sse,test_sse = gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)


# verbose=0
# gtgp.retrain_estimators(retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

KeyboardInterrupt: 

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    clf = DecisionTreeClassifier()
    clf.fit(X_train,y_train)

    num_trees = 1
    depth = clf.tree_.max_depth
    num_nodes = clf.tree_.node_count

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

    with open('./benchmark_DC/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# XGBoost

In [ ]:
import xgboost

In [ ]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=3,n_estimators=100)
    # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4)
    xgb = xgboost.XGBClassifier()
    xgb.fit(X_train,y_train)

    import json

    def item_generator(json_input, lookup_key):
        if isinstance(json_input, dict):
            for k, v in json_input.items():
                if k == lookup_key:
                    yield v
                else:
                    yield from item_generator(v, lookup_key)
        elif isinstance(json_input, list):
            for item in json_input:
                yield from item_generator(item, lookup_key)

    def tree_depth(json_text):
        json_input = json.loads(json_text)
        depths = list(item_generator(json_input, 'depth'))
        return max(depths) + 1 if len(depths) != 0 else 1

    train_acc = accuracy_score(y_train,xgb.predict(X_train))
    test_acc = accuracy_score(y_test,xgb.predict(X_test))
    train_f1 = f1_score(y_train,xgb.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,xgb.predict(X_test),average='macro')

    booster = xgb.get_booster()

    tree_df = booster.trees_to_dataframe()
    depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
    num_trees = len(depths)
    depth = np.average(depths)
    num_nodes = len(tree_df)

    with open('./benchmark_xgb/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# GDBT

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    clf = GradientBoostingClassifier()
    # clf = GradientBoostingClassifier()
    clf.fit(X_train,y_train)

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

    num_trees = len([ est for ests in clf.estimators_ for est in ests])
    depth = np.average([ max(1,est.tree_.max_depth) for ests in clf.estimators_ for est in ests])
    num_nodes = sum([ est.tree_.node_count for ests in clf.estimators_ for est in ests])

    with open('./benchmark_GBDT/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    rfc = RandomForestClassifier()
    rfc.fit(X_train,y_train)

    train_acc = accuracy_score(y_train,rfc.predict(X_train))
    test_acc = accuracy_score(y_test,rfc.predict(X_test))
    train_f1 = f1_score(y_train,rfc.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,rfc.predict(X_test),average='macro')

    num_trees = len(rfc.estimators_)
    depth = np.average([est.tree_.max_depth for est in rfc.estimators_])
    num_nodes = sum([est.tree_.node_count for est in rfc.estimators_])

    with open('./benchmark_RF/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)